In [3]:
import os
import pandas as pd
import os
import numpy as np
import cvxpy as cv
from multiprocessing import Pool
from functools import partial
cwd = os.getcwd()
cwd

'E:\\backtrader\\scripts'

In [4]:
data_path = r"E:\backtrader\CSMAR\周个股回报率文件\周个股回报率文件\weekly_return.csv"
def clean(path, data_type='panel'):
    assert data_type in ['panel', 'ts'], 'type must be panel or ts'
    df = pd.read_csv(path)
    df.dropna(inplace=True)
    df['id'] = df['id'].astype('str').str.zfill(6)
    if data_type == 'panel':
        return df
    else:
        df = df.pivot(index='date', columns='id', values='weekly_return')
        return df

df = clean(data_path, 'panel')
df

,id,date,close,Nday,weekly_return_divident,weekly_return
0,000001,1991-15,46.840,5,-0.029826,-0.029826
1,000001,1991-16,45.460,5,-0.029462,-0.029462
2,000001,1991-17,44.340,4,-0.024637,-0.024637
3,000001,1991-18,43.020,5,-0.029770,0.365088
4,000001,1991-19,41.750,5,-0.029521,-0.029521
...,...,...,...,...,...,...
3184373,900957,2023-38,0.379,5,-0.013021,-0.013021
3184374,900957,2023-39,0.401,4,0.058047,0.058047
3184375,900957,2023-41,0.387,5,-0.034915,-0.034913
3184376,900957,2023-42,0.354,5,-0.085270,-0.085271


In [5]:
df = clean(data_path, 'ts')
df

id,000001,000002,000003,000004,000005,000006,000007,000008,000009,000010,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
date,,,,,,,,,,,,,,,,,,,,,
1990-52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990-53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-02,NaN,NaN,NaN,NaN,-0.032169,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991-03,NaN,NaN,NaN,NaN,-0.023496,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-38,0.001783,-0.002235,NaN,0.083605,0.035714,0.000000,0.000000,-0.004082,-0.009643,-0.077586,...,-0.058824,0.037258,NaN,NaN,NaN,0.009804,0.055556,NaN,NaN,-0.013021
2023-39,-0.003559,-0.023152,NaN,-0.033755,0.000000,-0.029915,-0.021739,0.000000,0.014606,0.099688,...,0.033654,0.005029,NaN,NaN,NaN,0.004854,0.036011,NaN,NaN,0.058047
2023-41,-0.017857,-0.037462,NaN,0.119152,0.013793,-0.033040,0.004444,-0.028689,0.005758,0.056657,...,-0.032558,-0.003574,NaN,NaN,NaN,-0.004831,-0.048128,NaN,NaN,-0.034913


In [6]:
df_sample = df[df.index > '2022-26']
df_week = df_sample.dropna(axis=1)
df_week

id,000001,000002,000004,000005,000006,000007,000008,000009,000010,000011,...,900941,900942,900943,900945,900946,900947,900948,900952,900953,900957
date,,,,,,,,,,,,,,,,,,,,,
2022-27,0.048489,0.112323,-0.011853,0.011050,0.019912,0.080576,-0.004016,-0.014154,-0.089412,0.032787,...,-0.004673,0.165572,0.043933,0.015075,-0.028571,0.019380,0.041184,0.007692,0.179245,-0.034768
2022-28,-0.025469,-0.054902,0.075245,-0.092896,-0.043384,0.031957,-0.008065,-0.058148,-0.023256,-0.027967,...,0.091549,-0.030573,-0.016032,-0.009901,-0.021008,-0.007605,0.010507,-0.007634,0.088000,0.025729
2022-29,-0.089409,-0.086100,-0.161258,0.018072,-0.058957,0.018065,-0.032520,-0.069360,-0.074074,-0.139191,...,-0.035294,-0.001314,0.030550,-0.065000,-0.034335,-0.011494,0.516820,-0.003846,-0.110294,-0.055184
2022-30,-0.015257,-0.013621,0.091898,0.059172,0.012048,-0.086185,0.054622,-0.009009,0.051429,0.008662,...,0.021951,0.031579,0.019763,0.016043,0.026667,0.027132,0.007742,0.011583,0.008264,0.023009
2022-31,-0.010148,-0.009781,0.019934,0.022346,0.004762,0.022191,0.003984,0.176033,0.002717,0.019084,...,0.000000,-0.016582,-0.009690,0.005263,0.008658,0.003774,0.008323,0.000000,0.030738,0.010381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-38,0.001783,-0.002235,0.083605,0.035714,0.000000,0.000000,-0.004082,-0.009643,-0.077586,-0.009009,...,0.007653,-0.009777,0.002024,0.010204,0.000000,-0.058824,0.037258,0.009804,0.055556,-0.013021
2023-39,-0.003559,-0.023152,-0.033755,0.000000,-0.029915,-0.021739,0.000000,0.014606,0.099688,-0.013131,...,0.007595,0.012694,0.006061,0.030303,0.031447,0.033654,0.005029,0.004854,0.036011,0.058047
2023-41,-0.017857,-0.037462,0.119152,0.013793,-0.033040,0.004444,-0.028689,0.005758,0.056657,-0.037871,...,0.002513,-0.013928,0.008032,-0.044118,-0.012195,-0.032558,-0.003574,-0.004831,-0.048128,-0.034913


In [7]:
cov_matrix = df_week.cov().values
mu = df_week.mean().values
expected_return_of_portfolio = 0.0035
rf = 0.035/52
cov_matrix
# check if the matrix is symmetric
def symmetric_check(matrix):
    return matrix[np.where(np.isclose(matrix, matrix.T, atol=1e-8) == False)]
symmetric_check(cov_matrix)

array([], dtype=float64)

In [32]:
# 定义变量
weights = cv.Variable(len(mu))
#
# 定义目标函数
objective = cv.Minimize(cv.quad_form(weights, cov_matrix, assume_PSD=True))

# 定义约束条件
constraints = [
    cv.sum(weights) == 1,  # 权重之和为1
    weights.T @ mu == expected_return_of_portfolio,  # 期望收益率为给定值
    weights >= 0,  # 权重大于等于0
]

# 创建问题并求解
problem = cv.Problem(objective, constraints)
problem.solve()

# 输出结果
print("最优权重:", weights.value)
print("最小方差:", problem.value)


最优权重: [1.73453364e-07 7.78014431e-08 1.35867654e-07 ... 1.16763775e-07
 2.52205068e-07 2.82970317e-07]
最小方差: 1.5485014745242196e-05


In [33]:
threshold = 0.0001
weights.value[np.abs(weights.value) < threshold] = 0
weights.value[weights.value > 0]
portfolio_return = weights.value @ mu
portfolio_risk = problem.value ** 0.5
# 输出组合收益率，组合风险和夏普比率
print("组合收益率:", portfolio_return)
print("组合风险:", portfolio_risk)
print("夏普比率:", (portfolio_return - rf) / portfolio_risk)

组合收益率: 0.003501690536140696
组合风险: 0.003935100347544163
夏普比率: 0.7188161325616685


In [34]:
print(mu.max())
mu.min()

0.03106946268656717


-0.019235641791044776

In [8]:
e_range = np.linspace(mu.min(), mu.max(), 1000)
def solve(e, cov, mu_vector):
    import cvxpy
    weights = cvxpy.Variable(len(mu_vector))
    objective = cvxpy.Minimize(cvxpy.quad_form(weights, cov, assume_PSD=True))
    constraints = [
        cvxpy.sum(weights) == 1,  # 权重之和为1
        weights.T @ mu_vector == e,  # 期望收益率为给定值
        weights >= 0,  # 权重大于等于0
    ]
    problem = cvxpy.Problem(objective, constraints)
    problem.solve()
    return problem.value ** 0.5

partial_solve = partial(solve, cov=cov_matrix, mu_vector=mu)


In [9]:
risk = partial_solve(0.0035)

In [10]:
risk

0.003935100347544163

In [ ]:
with Pool(10) as p:
    risks = p.map(partial_solve, e_range)
risks